# 04 LSTM Eğitimi

In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.config import PROJECT_ROOT
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split


In [2]:
df = pd.read_parquet(PROJECT_ROOT.joinpath('data/interim/sample.parquet'))
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1

data_array = df.values


In [6]:
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=10, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=10)


In [7]:
sensors = data_array.shape[1]
predictor = TrafficPredictor('lstm', sensors, X_STEP, Y_STEP)
predictor.train(train_loader, val_loader, epochs=50, 
                save_path=PROJECT_ROOT.joinpath("models/lstm.pt"), 
                save_optimizer=True)
predictor.show()


  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 1/50  Train Loss: 15510.4795  Val Loss: 15245.0293  Val MSE: 15245.0303  Val R2: -1.6512
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 2/50  Train Loss: 15509.3027  Val Loss: 15243.8008  Val MSE: 15243.8027  Val R2: -1.6510
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 3/50  Train Loss: 15508.1074  Val Loss: 15242.4121  Val MSE: 15242.4111  Val R2: -1.6508
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 4/50  Train Loss: 15506.7520  Val Loss: 15240.8389  Val MSE: 15240.8408  Val R2: -1.6505
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 5/50  Train Loss: 15505.2109  Val Loss: 15239.0625  Val MSE: 15239.0635  Val R2: -1.6502
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt